In [3]:
using Revise

In [38]:
using RigidBodyDynamics
using RigidBodyDynamics: Contact
using MeshCatMechanisms
using MeshCat
using StaticArrays
using Blink: Window
using OSQP.MathOptInterfaceOSQP
using QPControl
import MathOptInterface
const MOI = MathOptInterface
using RigidBodySim
using Gurobi

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/Gurobi.ji for module Gurobi.


In [5]:
vis = Visualizer()
open(vis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 7335`, ProcessRunning), TCPSocket(RawFD(57) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(59) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 1, Nullable{Any}(true))))

In [118]:
function defaultoptimizer()
    optimizer = OSQPOptimizer()
    MOI.set!(optimizer, OSQPSettings.Verbose(), false)
    MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set!(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    MOI.set!(optimizer, OSQPSettings.WarmStart(), false)
    optimizer
#     GurobiOptimizer(OutputFlag=0)
end

defaultoptimizer (generic function with 1 method)

In [119]:
world = RigidBody{Float64}("world")
hopper = Mechanism(world; gravity=SVector(0, 0, -9.81))

frame = CartesianFrame3D("core")
inertia = SpatialInertia(frame, SDiagonal(1., 1, 1), SVector(0., 0, 0), 10.)
core = RigidBody(inertia)
joint = Joint("floating_base", Prismatic(SVector(0., 0, 1)))
attach!(hopper, world, core, joint)
position_bounds(joint) .= RigidBodyDynamics.Bounds(0, 10)
velocity_bounds(joint) .= RigidBodyDynamics.Bounds(-1000, 1000)
effort_bounds(joint) .= RigidBodyDynamics.Bounds(0, 0)

frame = CartesianFrame3D("foot")
inertia = SpatialInertia(frame, SDiagonal(0.05, 0.05, 0.05), SVector(0., 0, 0), 1.0)
foot = RigidBody(inertia)
joint = Joint("foot_extension", Prismatic(SVector(0., 0, 1)))
attach!(hopper, core, foot, joint)
position_bounds(joint) .= RigidBodyDynamics.Bounds(-1., -0.5)
velocity_bounds(joint) .= RigidBodyDynamics.Bounds(-1000, 1000)
effort_bounds(joint) .= RigidBodyDynamics.Bounds(-200, 200)


floor = HalfSpace3D(Point3D(default_frame(world), 0., 0, 0), FreeVector3D(default_frame(world), 0., 0, 1))
add_environment_primitive!(hopper, floor)
contactmodel = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))
add_contact_point!(foot, Contact.ContactPoint(Point3D(default_frame(foot), 0., 0, 0), contactmodel))

In [120]:
function initialize!(state::MechanismState)
    set_configuration!(state, [1, -1])
    set_velocity!(state, [0, 0])
    set_additional_state!(state, zeros(Float64, num_additional_states(state)))
    state
end

initialize! (generic function with 1 method)

In [121]:
delete!(vis)
mvis = MechanismVisualizer(hopper, Skeleton(randomize_colors=true), vis)
copy!(mvis, initialize!(MechanismState(hopper)))

In [122]:
Δt = 0.01

mpc_controller = QPControl.MPCController{4}(hopper, defaultoptimizer())

stage = QPControl.addstage!(mpc_controller, Δt)
for surface in hopper.environment.halfspaces
    for body in bodies(hopper)
        for contact_point in contact_points(body)
            controller_contact = QPControl.addcontact!(stage,
                body,
                location(contact_point),
                FreeVector3D(default_frame(body), 0., 0, 1),
                0.8,
                surface
            )
            controller_contact.maxnormalforce = 1e9
        end
    end
end

mpc_controller.running_state_cost.Q .= Diagonal([100., 1., 1., 0.1])
mpc_controller.running_state_cost.x0 .= [1., -1., 0., 0.]
mpc_controller.running_input_cost.Q .= Diagonal([0.01, 0.01])

2×2 Array{Float64,2}:
 0.01  0.0 
 0.0   0.01

In [123]:
state = initialize!(MechanismState(hopper))
τ = similar(velocity(state))
mpc_controller(τ, 0.0, state)
τ

2-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
   0.000477916
 -98.0833     

In [124]:
state = initialize!(MechanismState(hopper))
set_configuration!(state, [0.5, -0.5])
controller = PeriodicController(similar(velocity(state)), Δt, mpc_controller)
dynamics = Dynamics(hopper, controller)
problem = ODEProblem(dynamics, state, (0., 10.0))
sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6, force_dtmin=true)
setanimation!(mvis, sol)

true

ERROR (unhandled task failure): OutOfMemoryError()
Stacktrace:
 [1] setindex!(::ObjectIdDict, ::Any, ::Any) at ./associative.jl:430
 [2] preserve_handle at ./libuv.jl:51 [inlined]
 [3] stream_wait(::Timer, ::Condition, ::Vararg{Condition,N} where N) at ./stream.jl:40
 [4] wait(::Timer) at ./event.jl:357
 [5] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:430
